In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
from sklearn.metrics import roc_auc_score

# Configuración del dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Directorios
train_dir = '/kaggle/input/pneumonia-sai3/pneumonia-kaggle/train/'
test_dir = '/kaggle/input/pneumonia-sai3/pneumonia-kaggle/test/'

# Parámetros
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_EPOCHS = 30
LEARNING_RATE = 0.001

# Transformaciones
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Cargar datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

# Dataset personalizado para imágenes no clasificadas
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.image_filenames[idx]

# Dataset de prueba
test_dataset = TestDataset(image_dir=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Clase del modelo CNN
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self._initialize_fc_input()

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.fc_input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def _initialize_fc_input(self):
        dummy_input = torch.zeros(1, 3, *IMG_SIZE)
        output = self.conv_layers(dummy_input)
        self.fc_input_dim = output.view(-1).shape[0]

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Inicialización del modelo
model = PneumoniaCNN().to(device)

# Función de pérdida y optimizador
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

Using device: cuda


In [2]:
# Evaluación del modelo (con AUROC)
def evaluate_model_auroc(model, loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze()
            all_predictions.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcular AUROC
    auroc = roc_auc_score(all_labels, all_predictions)
    return auroc

# Entrenamiento del modelo con AUROC
def train_model_auroc(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_predictions.extend(outputs.cpu().detach().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Cálculo de AUROC en entrenamiento
        train_auroc = roc_auc_score(all_labels, all_predictions)

        # Evaluación en validación
        val_auroc = evaluate_model_auroc(model, val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
              f"Train AUROC: {train_auroc:.4f}, Val AUROC: {val_auroc:.4f}")

In [3]:
# Entrenamiento
train_model_auroc(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS)



Epoch 1/30, Loss: 0.9604, Train AUROC: 0.8960, Val AUROC: 0.9887
Epoch 2/30, Loss: 0.1435, Train AUROC: 0.9832, Val AUROC: 0.9932
Epoch 3/30, Loss: 0.1470, Train AUROC: 0.9823, Val AUROC: 0.9957
Epoch 4/30, Loss: 0.1458, Train AUROC: 0.9825, Val AUROC: 0.9946
Epoch 5/30, Loss: 0.1355, Train AUROC: 0.9857, Val AUROC: 0.9964
Epoch 6/30, Loss: 0.1111, Train AUROC: 0.9897, Val AUROC: 0.9967
Epoch 7/30, Loss: 0.0975, Train AUROC: 0.9920, Val AUROC: 0.9953
Epoch 8/30, Loss: 0.1098, Train AUROC: 0.9895, Val AUROC: 0.9977
Epoch 9/30, Loss: 0.0964, Train AUROC: 0.9925, Val AUROC: 0.9942
Epoch 10/30, Loss: 0.1040, Train AUROC: 0.9912, Val AUROC: 0.9975
Epoch 11/30, Loss: 0.0939, Train AUROC: 0.9923, Val AUROC: 0.9976
Epoch 12/30, Loss: 0.0880, Train AUROC: 0.9936, Val AUROC: 0.9974
Epoch 13/30, Loss: 0.0891, Train AUROC: 0.9934, Val AUROC: 0.9972
Epoch 14/30, Loss: 0.0893, Train AUROC: 0.9931, Val AUROC: 0.9983
Epoch 15/30, Loss: 0.0878, Train AUROC: 0.9937, Val AUROC: 0.9977
Epoch 16/30, Loss: 

In [4]:
# Predicción en el conjunto de prueba
model.eval()
test_results = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images).squeeze()
        probability = outputs.item()
        test_results.append({'Id': filenames[0], 'Category': probability})

# Guardar las predicciones en un archivo CSV
results_df = pd.DataFrame(test_results)
results_df.to_csv('predictions0.csv', index=False)

In [5]:
# Guardar solo los pesos
torch.save(model.state_dict(), 'model_weights.pth')

# Guardar el modelo completo
torch.save(model, 'model_complete.pth')